# Libbitcoin System (C++) : Mnemonics & HD Wallets
In this tutorial, we create mnemonic backup phrases, from which HD private and public keys can be derived for use. The mnemonic phrase can later be used to recover hd keys used in a previous wallet with the same seed.

## 1) Seeding an HD wallet.
<img src="images/hd_mnemonic_to_master_cpp.jpg" alt="drawing" style="" width="800px"/>



### Libbitcoin-System (C++) Setup

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System
#include <bitcoin/bitcoin.hpp> 

### 1.1) Deriving the mnemonic phrase (BIP39)

In [ ]:
bc::data_chunk my_entropy(16); // 16 bytes = 128 bits
bc::pseudo_random_fill(my_entropy);


In [ ]:
bc::wallet::word_list my_word_list = bc::wallet::create_mnemonic(
  my_entropy, bc::wallet::language::ja);
std::cout << bc::join(my_word_list) << std::endl; //join to a single string with spaces


In [ ]:
std::string my_word_list_literal =
  "むける りりく あんぜん ひろい よかぜ いっぽう でぬかえ むいか うんてん げいのうじん ひほう きぞく";
my_word_list = bc::split(my_word_list_literal, " ", true);

// mnemonic-to-seed always derives a 512-bit long seed.
auto hd_seed = bc::wallet::decode_mnemonic(my_word_list);

std::cout << bc::encode_base16(bc::to_chunk(hd_seed)) << std::endl;


## 1.2) Deriving the master HD keys (BIP32)

In [ ]:
bc::wallet::hd_private m(bc::to_chunk(hd_seed), bc::wallet::hd_private::testnet);
std::cout << m.encoded() << std::endl;


In [ ]:
auto M = m.to_public();
std::cout << M.encoded() << std::endl;


## 2) Deriving HD children
<img src="images/hd_children_derivation_cpp.jpg" alt="drawing" style="" width="800px"/>

### 2.1) Deriving unhardened children hd keys.

**`m` &#8658; `m/0/1/2`** `(private-key child derivation)`

In [ ]:
auto m_012 = m.derive_private(0).derive_private(1).derive_private(2);
std::cout << m_012.encoded() << std::endl;


**`M` &#8658; `M/0/1/2`** `(public-key child derivation)`

In [ ]:
auto M_012 = M.derive_public(0).derive_public(1).derive_public(2);
std::cout << M_012.encoded() << std::endl;


**`m/0/1/2` &#8658; `M/0/1/2`** 

In [ ]:
auto M_012_ = m_012.to_public();
std::cout <<(M_012 == M_012_) << std::endl;


**Payment addresses from hd-keys**

In [ ]:
bc::ec_compressed M_012_point(M_012);
bc::wallet::ec_public M_012_public(M_012_point);
auto M_012_public_address = M_012_public.to_payment_address(
  bc::wallet::payment_address::testnet_p2kh);

std::cout << M_012_public_address.encoded() << std::endl;


### 2.2) Deriving hardened children hd keys.

**`m` &#8658; `m/44'`** 

In [ ]:
auto m_44h = m.derive_private(44 + bc::wallet::hd_first_hardened_key);
std::cout << m_44h.encoded() << std::endl;


**`m` &#8658; `m/44'` &#8658; `M/44'`** 

In [ ]:
auto M_44h = m.derive_private(44 + bc::wallet::hd_first_hardened_key).to_public();
std::cout << M_44h.encoded() << std::endl;


**`m/44'` &#8658; `m/44'/1'` &#8658; `M/44'/1'`** 

In [ ]:
auto M_44h_1h = m_44h.derive_private(1 + bc::wallet::hd_first_hardened_key).to_public();
std::cout << M_44h_1h.encoded() << std::endl;


**Try: `M/44'` &#8658; `M/44'/1'`**

In [ ]:
auto M_44h_1h_ = M_44h.derive_public(1 + bc::wallet::hd_first_hardened_key);
std::cout << M_44h_1h_.encoded() << std::endl;
